# Log any table with `mlflow.log_table()`

The `mlflow.evaluate()` feature provides a very useful tool for evaluating mlflow models on specified datasets and logging the metrics and artifacts to MLFlow tracking.

`mlflow.log_table()` provides a flexible option for logging arbitrary tables in cases where `mlflow.evaluate()` isn't the right fit. For example, you may want to test an LLM on an evaluation dataset and log the results without saving/loading it as an MLFlow model.

This example shows how to log a table of model outputs to MLFlow tracking without logging the model or invoking `mlflow.evaluate`.

In [ ]:
import pandas as pd
import mlflow
from transformers import pipeline

## Load a Hugging Face Model

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

pipe = pipeline("text-generation", model="tiiuae/falcon-7b-instruct", trust_remote_code=True,
                device_map="auto")